In [ ]:
!pip install pandas

In [ ]:
!pip install psycopg2 #install postgres driver

In [ ]:
DB_HOST = '89.145.163.87'
DB_PORT = 5432
DB_USERNAME = 'group11'
DB_PASSWORD = 'tsw2020'
DB_NAME = 'tweets'

In [ ]:
import psycopg2
import pandas as pd

# connect to DB
conn = psycopg2.connect(host=DB_HOST, port=DB_PORT, database=DB_NAME, user=DB_USERNAME, password=DB_PASSWORD)
cur = conn.cursor()

sql_query = """
SELECT
tweet_id,
inserted,
tweet_body->'created_at' as created_at,
tweet_body->'user'->'id_str' as user_id_str,
tweet_body->'user'->'screen_name' as user_screen_name,
tweet_body->'user'->'followers_count' as followers_count,

tweet_body->'entities'->'user_mentions'->0->>'screen_name' as user_mention_screen_name,
tweet_body->'entities'->'user_mentions'->0->>'id' as user_mention_id_str,

tweet_body->'in_reply_to_status_id' as in_reply_to_status_id,
tweet_body->'in_reply_to_status_id_str' as in_reply_to_status_id_str,
tweet_body->'in_reply_to_user_id_str' as in_reply_to_user_id_str,
tweet_body->'in_reply_to_screen_name' as in_reply_to_screen_name,

tweet_body->'retweeted_status'->'user'->>'id_str' as retweeted_user_id_str,
tweet_body->'retweeted_status'->'user'->>'screen_name' as retweeted_user_screen_name,

tweet_body->'metadata'->'result_type' as result_type,
tweet_body->>'text' as text
FROM tweet
;
"""
tweets_database = pd.read_sql_query(sql_query, conn)
tweets_database.set_index('tweet_id')

print("Dataframe shape: {}".format(tweets_database.shape))
print("-"*20)
tweets_database.head(5)
print("-"*20)
print(tweets_database.info(memory_usage=True))

cur.close()
conn.close()

# write the serialized object to a file
tweets_database.to_pickle(path='tweets_database.pickle')